In [47]:
import os
import requests

In [48]:
def EsEntero(num):
    try:
        k = int(num)
        return k
    except:
        return -1

In [49]:
# EsEntero("ssss")

In [50]:
def PintarMenu(listaOpc):
    seguir = True
    while(seguir):
        for i,opcion in enumerate(listaOpc):
             print(f"{i}.-{opcion}")
        cad = input("Inserte una opción:")
        k = EsEntero(cad)
        if 0<=k and k< len(listaOpc):
            return k 
        elif k == -1:
            print("\nDebe insertar un número entero.\n")
        else:
            print("\nEl número debe estar entre 0 y 5.\n")

In [51]:
def main():
    print("Programa de la práctica 1\nTrabajo realizado por : \n\tMaría Mateos López \n\tAndrea Sayago Butrón\n Menú ")
    
    seguir = True
    while seguir:
        listaOpciones = ["Salir",
                        "Introducir API key",
                        "Mostrar el listado de estaciones por provincia",
                        "Consultar datos actuales de una estación", 
                        "Calcular medias de valores de estaciones por provincia",
                        "Consultar predicciones meteorológicas de municipios"]
        k = PintarMenu(listaOpciones)
        print(f"\nOpción elegida: {listaOpciones[k]}\n")
        match k:
            case 0:
                seguir = False
            case 1:
                pass
                # PediraApiKey
            case 2:
                pass
                # sldfjlsd
            case 4:
                pass
            case 5:
                pass
            

In [52]:
if __name__ == '__main__':
    main()

Programa de la práctica 1
Trabajo realizado por : 
	María Mateos López 
	Andrea Sayago Butrón
 Menú 
0.-Salir
1.-Introducir API key
2.-Mostrar el listado de estaciones por provincia
3.-Consultar datos actuales de una estación
4.-Calcular medias de valores de estaciones por provincia
5.-Consultar predicciones meteorológicas de municipios

Opción elegida: Introducir API key

0.-Salir
1.-Introducir API key
2.-Mostrar el listado de estaciones por provincia
3.-Consultar datos actuales de una estación
4.-Calcular medias de valores de estaciones por provincia
5.-Consultar predicciones meteorológicas de municipios

El número debe estar entre 0 y 5.

0.-Salir
1.-Introducir API key
2.-Mostrar el listado de estaciones por provincia
3.-Consultar datos actuales de una estación
4.-Calcular medias de valores de estaciones por provincia
5.-Consultar predicciones meteorológicas de municipios

Debe insertar un número entero.

0.-Salir
1.-Introducir API key
2.-Mostrar el listado de estaciones por provinc